<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install sodapy

In [0]:
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [100]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=2000000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16T16:15:00.000,2019-01-16T16:30:00.000,1346,8.21715513808,7,77,15,0,2.55,17.55
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01T06:00:00.000,2018-11-01T07:00:00.000,3113,20.4166496394989,45,23,17.5,0,2.5,20
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24T19:45:00.000,2019-01-24T19:45:00.000,469,1.5670652572791,31,31,5,0,2.55,7.55
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21T18:15:00.000,2018-12-21T18:30:00.000,880,4.1697883216,69,39,10,0,0,10
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05T14:45:00.000,2019-03-05T15:00:00.000,1506,7.32213988795855,28,29,10,0,2.55,12.55
...,...,...,...,...,...,...,...,...,...,...,...
1999995,1143f37b3053e2f9d4f37f613cdd5825c26dda78,2019-08-12T01:45:00.000,2019-08-12T02:00:00.000,1015,6.722369,5,8,12.5,3,2.55,18.05
1999996,1143f38297b979a3b88696b2244bd02341d2a500,2019-07-19T09:15:00.000,2019-07-19T09:30:00.000,1306,7.388153,32,6,15,5,2.55,22.55
1999997,1143f48e90eb89cccbff886fc4147e3c5a27fa35,2019-09-03T16:15:00.000,2019-09-03T17:00:00.000,2786,14.704924,77,76,27.5,0,7.55,35.05
1999998,1143f5069330b99ec01ae6e2da8e24836719f852,2019-09-29T18:15:00.000,2019-09-29T18:45:00.000,1753,23.44713608976,76,NaN,27.5,0,9.14,36.64


In [97]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

def set_precip(precip):
    if precip not in ['rain', 'snow']: 
        precip = 'clear'

    return precip

weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precip
0,ecc1cd019a7245f5d1ee0b6a6de54a7a747891c3,2019-03-11 08:45:00,2019-03-11 08:45:00,735,2.853204,8,7.5,0.0,2.55,10.05,0,8,2,2019-03-11 08:00:00,clear
1,ecc1cd815dbee6b5f9cf91e5d8209500e8071853,2019-03-27 19:30:00,2019-03-27 19:45:00,1015,6.769226,28,10.0,0.0,0.00,10.00,2,19,6,2019-03-27 19:00:00,clear
2,ecc1cde0a29114f08b85de6ebd7a5f2f72a52ea1,2018-11-18 15:00:00,2018-11-18 15:15:00,753,3.446462,28,7.5,0.0,2.50,10.00,6,15,5,2018-11-18 15:00:00,clear
3,ecc1cf1cbb2db0c04cf39479d018b2688baf7268,2019-01-23 18:45:00,2019-01-23 19:00:00,892,2.953652,69,7.5,0.0,0.00,7.50,2,18,6,2019-01-23 18:00:00,clear
4,ecc1cf5253f0aefc61e1bbb83109fe5a185316f7,2018-12-14 18:45:00,2018-12-14 19:30:00,2512,10.672421,NaN,15.0,0.0,0.00,15.00,4,18,6,2018-12-14 18:00:00,clear


In [99]:
columns_to_use = ['pickup_community_area', 'precip']

columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X_cat = df.drop(columns=columns_to_drop)
X_cat = X_cat.fillna('0')


enc = OneHotEncoder()

y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.25)

# Transform training and test sets
X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train.columns)
X_train_df = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

X_train_df = sm.add_constant(X_train_df)
model = sm.OLS(y_train,X_train_df).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.8411
Date:                Wed, 22 Jan 2020   Prob (F-statistic):              0.499
Time:                        14:38:34   Log-Likelihood:            -5.6982e+05
No. Observations:              150000   AIC:                         1.140e+06
Df Residuals:                  149995   BIC:                         1.140e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -4.07e+12   4.29e+12     -0.948      0.343   -1.25e+13    4.35e+12
precip_0       4.07e+12   4.29e+12      0.948      0.343   -4.35e+12    1.25e+13
precip_clear   4.07e+12   4.29e+12      0.948      0.343   -4.35e+12    1.25e+13
precip_rain    4.07e+12   4.29e+12      0.948      0.343   -4.35e+12    1.25e+13
precip_snow    4.07e+12   4.29e+12      0.948      0.343   -4.35e+12    1.25e+13
==============================================================================
Omnibus:                   133035.874   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         14763957.466
Skew:                           3.811   Prob(JB):                         0.00
Kurtosis:                      51.001   Cond. No.                     4.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.27e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [27]:

# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_df, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_df, y_train))
print('Test r^2:', linreg.score(X_test_df, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_df)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_df)))


Training r^2: 0.31120337885173954
Test r^2: 0.3158165059278203
Training MSE: 108.06877067877224
Test MSE: 104.45088889944266


In [71]:
precip_df.loc[precip_df['precip'] == 'rain']
precip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 2 columns):
start_date_plus_hour    10223 non-null datetime64[ns]
precipitation           10223 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 159.9+ KB
